# Tile Generation from GeoTIFF Images

In this notebook, a Python script designed to facilitate the generation of smaller tiles from large GeoTIFF images. The process involves partitioning the images into manageable sections, often referred to as tiles, which can be useful for various applications such as processing large datasets in smaller chunks or serving imagery over the web efficiently.

In [10]:
# import all necessary libraries
import os
from itertools import product
import rasterio as rio
from rasterio import windows

Now we will read GeoTIFF images from a specified input directory (in_path), divide them into smaller tiles, and save these tiles to an output directory (out_path).

In [11]:
in_path = 'data'
# input_filename = 'image.tif'
 
out_path = 'output_data'
# output_filename = 'tile_{}-{}.tif'
w=0
def get_tiles(ds, width=5, height=5):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform

image_files = [f for f in os.listdir(in_path) if f.endswith('.tif')]
for file in image_files:
    with rio.open(os.path.join(in_path,file)) as inds:
        tile_width, tile_height =1024,1024

        meta = inds.meta.copy()

        for window, transform in get_tiles(inds,tile_width, tile_height):
            print(window)
            meta['transform'] = transform
            meta['width'], meta['height'] = window.width, window.height
            outpath = os.path.join(out_path,format(w)+file.format(int(window.col_off), int(window.row_off)))
            with rio.open(outpath, 'w', **meta) as outds:
                outds.write(inds.read(window=window))
                w=w+1

Window(col_off=0, row_off=0, width=1024, height=1024)
Window(col_off=0, row_off=1024, width=1024, height=1024)
Window(col_off=0, row_off=2048, width=1024, height=1024)
Window(col_off=0, row_off=3072, width=1024, height=1024)
Window(col_off=0, row_off=4096, width=1024, height=1024)
Window(col_off=0, row_off=5120, width=1024, height=1024)
Window(col_off=0, row_off=6144, width=1024, height=1024)
Window(col_off=0, row_off=7168, width=1024, height=1024)
Window(col_off=0, row_off=8192, width=1024, height=603)
Window(col_off=1024, row_off=0, width=1024, height=1024)
Window(col_off=1024, row_off=1024, width=1024, height=1024)
Window(col_off=1024, row_off=2048, width=1024, height=1024)
Window(col_off=1024, row_off=3072, width=1024, height=1024)
Window(col_off=1024, row_off=4096, width=1024, height=1024)
Window(col_off=1024, row_off=5120, width=1024, height=1024)
Window(col_off=1024, row_off=6144, width=1024, height=1024)
Window(col_off=1024, row_off=7168, width=1024, height=1024)
Window(col_off

In [13]:
import os
import rasterio as rio
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.io import MemoryFile

# Define the directory containing the chunks
chunks_dir = 'output_georeferenced'

# Get list of all chunk files
chunk_files = [os.path.join(chunks_dir, f) for f in os.listdir(chunks_dir) if f.endswith('.tif')]

# Read all chunks into a list of Rasterio datasets
datasets = [rio.open(chunk) for chunk in chunk_files]

# Merge datasets into a single image
mosaic, out_trans = merge(datasets)

# Copy metadata from the first dataset
out_meta = datasets[0].meta.copy()

# Update metadata with new dimensions, transform, and other properties
out_meta.update({
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_trans,
    "count": mosaic.shape[0]
})

# Define the path for the output file
output_file = 'data/merged_image.tif'

# Write the mosaic to a new GeoTIFF file
with rio.open(output_file, 'w', **out_meta) as dest:
    dest.write(mosaic)

# Close all datasets
for dataset in datasets:
    dataset.close()

print(f'Merged image saved to {output_file}')


Merged image saved to data/merged_image.tif
